We would've seen most of these iteration tools in the previous sections and you'll recognise that they're ***all*** **lazy iterators** and not **iterables**, making them highly efficient.

# 01 - Aggregators

#### Lecture 

Aggregators are functions that iterate through an iterable and returns a single value that (usually) takes into account every element of the iterable.

For example `min(iterable)`, `max(iterable)`, `sum(iterable)`

Also: `any(iterable)` and `all(iterable)`. Regarding these two, remember that **every** object has a truth value. The rule for all objects is the following:

**Every object has a **`True`** truth value, **except**:

- None
- False
- 0 in any numeric type (e.g. int, float etc)
- empty sequences (e.g. list, tuple, string)
- empty mapping types (e.g. dictionaries, sets)
- custom classes that implement a `__bool__` or `__len__` method that returns False or 0. If neither is present, we default to `True`.

**Definition**

Predicate: A predicate is a function that takes a single argument and returns `True` or `False`. For example, `bool()`.

We can make `all()` and `any()` more useful by first **applying a predicate** to each element of the iterable.

#### Example 1

For example, say we want to know if **every element is less that 10**:

In [18]:
l = [1, 2, 3, 4, 100, 5, 6]

pred = lambda x: x < 10

result = [pred(item) for item in l]

all(result)

False

The neater way to do this is using `map(fn, iterable)` which applies a **predicate** to each element in the iterable:

In [19]:
def pred_new(x):
    print(f'Mapping {x}', end=', ')
    return x < 10

all(map(pred_new, l))

Mapping 1, Mapping 2, Mapping 3, Mapping 4, Mapping 100, 

False

Because `map` is an iterator, it doesn't have to map every element first and then pass the result to `all()`. Instead, `all()` will lazily request a result from `map`; if it receives a `False` it will terminate immediately. This is why we don't see the final two elements (5 and 6) of `l` being printed.

If we do not want to use `map`, we should use a `generator comprehension`, *NOT* a list comprehension, so that we get the benefits of lazy evaluation.

In [20]:
result = (pred_new(item) for item in l)
all(result)

Mapping 1, Mapping 2, Mapping 3, Mapping 4, Mapping 100, 

False

#### Example 2

We have a file call `car-brands.txt`. We want to know if **every** car name is longer than 3 (4 including the \n character at the end):

Recall that `open(<file>)` (or `f`) is a lazy iterator - we don't need to loop through each line. We can pass `f` to another iterator such as `map`, and pass that output to another iterator such as `all` or `any` and they'll terminate at the right time. 

In [26]:
filename = '../Section 08 - Iteration Tools/01 - Aggregators/car-brands.txt'

with open(filename) as f:
    result = all(map(lambda row: len(row) >= 4, f))
    print(result)

True


# 02 - Slicing Iterables

Recall that we could slice sequences with `[]` notation as well as the `slice` object:

In [32]:
seq = list(range(0, 10, 1))
print(seq[2:8:2])
print(seq[slice(2, 8, 2)])

[2, 4, 6]
[2, 4, 6]


But we can slice **iterables** (including **iterators**) with `islice` from `itertools`:
```python
islice(iterable, start, stop, step)
```

- `islice` will iterate through the iterable until it has met the conditions of the slice. For example, if we only want a slice of the first 5 objects of an infinite iterable, it will raise the `StopIteration` error after the 5th element
- Recalling that all itertools are **iterators**, `islice()` will *yield* a value, not return it, and hence `islice` returns a **lazy iterator**.

In the example below `factorials()` is an infinite iterable (more specifically an iterator), not a sequence, so it cannot be sliced regularly - we need to use `islice`. 

In [46]:
import math
from itertools import islice

def factorials():
    idx = 0
    while True:
        yield math.factorial(idx)
        idx += 1

result = islice(factorials(), 2, 9, 2)
list(result)

[2, 24, 720, 40320]

We've exhausted the `islice` iterator, so we can't reuse it.

In [41]:
list(result)

[]

# 03 - Selecting and Filtering

#### `filter`

The `filter` function takes an iterable and applies a predicate to it. If the predicate returns `True`, it will retain that element; otherwise, it'll throw it away.
```python
filter(predicate, iterable)
```

The equivalent way to get the same functionality of `filter` is with the following:
```python
(item for item in iterable if pred(item))
```

##### Example 1

In [1]:
l = [2, 1, 10, 5, 3, 6, 1, 10]
result = filter(lambda x: x < 4, l)
result

In [2]:
list(result)

[2, 1, 3, 1]

##### Example 2

In this example, we are going to iterate through a list of cube values and throw away all the values that are even, but we're going to do it lazily.

In [15]:
def gen_cubes(n):
    for i in range(n):
        print(f'yielding {i}')
        yield i**3

In [16]:
def is_odd(x):
    return x % 2 == 1

In [17]:
filtered = filter(is_odd, gen_cubes(10))

In [18]:
list(filtered)

yielding 0
yielding 1
yielding 2
yielding 3
yielding 4
yielding 5
yielding 6
yielding 7
yielding 8
yielding 9


[1, 27, 125, 343, 729]

##### `filterfalse` 

This is not builtin but it's in the standard library. It does what it says on the tin:

##### Example 1

In [3]:
from itertools import filterfalse

l = [2, 1, 10, 5, 3, 6, 1, 10]
result = filterfalse(lambda x: x < 4, l)
list(result)

[10, 5, 6, 10]

##### Example 2

This is the same as the example in `filter` but the inverse.

In [19]:
def gen_cubes(n):
    for i in range(n):
        print(f'yielding {i}')
        yield i**3

In [20]:
def is_odd(x):
    return x % 2 == 1

In [21]:
filtered = filterfalse(is_odd, gen_cubes(10))

In [22]:
list(filtered)

yielding 0
yielding 1
yielding 2
yielding 3
yielding 4
yielding 5
yielding 6
yielding 7
yielding 8
yielding 9


[0, 8, 64, 216, 512]

#### `compress`

This is not a compressor in the sense of say a zip archive.

It is basically a way of *filtering* one iterable, using the truthiness of items in another iterable, pairwise.

```python
data =      ['a',   'b', 'c', 'd',   'e']
              ^      ^    ^    ^      ^
              |      |    |    |      |
selectors = [True, False, 1,   0]  # None

compress(data, selectors) -> 'a', 'c'
```
Since the first and third element of `selectors` are truthy, we will only yield the first and third elements of `data`.

In [23]:
from itertools import compress

data = ['a', 'b', 'c', 'd', 'e']
selectors = [True, False, 1, 0]

compressed = compress(data, selectors)
list(compressed)

['a', 'c']

#### `takewhile`

```python
takewhile(pred, iterable)
```
This function returns an iterator that will yield while `predicate(item)` is Truthy. Once we run into a Falsy value, the iterator becomes exhausted.

##### Example 1

In [4]:
from itertools import takewhile

result = takewhile(lambda x: x < 5, [1, 2, 10, 3, 4])

for i in result:
    print(i)

1
2


##### Example 2

In this example, we will calculate numerous `sin(x)` values for evenly spaced `x`. 

In [11]:
from math import sin, pi

def sine_wave(n):
    start = 0
    max_ = 2 * pi
    step = (max_ - start) / (n-1)
    for _ in range(n):
        yield round(sin(start), 2)
        start += step    

In [12]:
list(sine_wave(15))

[0.0,
 0.43,
 0.78,
 0.97,
 0.97,
 0.78,
 0.43,
 0.0,
 -0.43,
 -0.78,
 -0.97,
 -0.97,
 -0.78,
 -0.43,
 -0.0]

In [13]:
from itertools import takewhile

list(takewhile(lambda x: 0 <= x <= 0.9, sine_wave(15)))

[0.0, 0.43, 0.78]

#### `dropwhile`

```python
dropwhile(pred, iterable)
```
This function is the inverse of the above. It returns an iterator that will start iterating and `yield` *all* remaining items unconditionally only once `predicate(item)` becomes Falsy.

##### Example 1

In [5]:
from itertools import dropwhile

result = dropwhile(lambda x: x < 5, [1, 2, 10, 3, 4])

for i in result:
    print(i)

10
3
4


# 04 - Infinite Iterators

#### `itertools.count`

This is a lazy iterator similar to range as it has `start`, `step` but no `stop`.

Also, `start` and `step` do not have to be integers unlike with `range()` - they can be any numeric type.

As these iterators are infinite, it can be quite useful to pair them with a `takewhile` so that we can control when they stop

##### Example 1

In [2]:
from itertools import count, cycle, repeat, islice

g = count(10)

list(islice(g, 5))

[10, 11, 12, 13, 14]

Remember, `count` is infinite so we can't `list(g)`.

##### Example 2

In [8]:
from decimal import Decimal

g = count(Decimal('1.0'), Decimal('0.2'))

list(islice(g, 5))

[Decimal('1.0'),
 Decimal('1.2'),
 Decimal('1.4'),
 Decimal('1.6'),
 Decimal('1.8')]

#### `itertools.cycle`

This cycles over a finite iterable (including iterators) indefinitely.
```python
cycle(['a', 'b', 'c']) -> 'a', 'b', 'c', 'a', 'b', 'c', 'a', ...
```
**One important thing to note:** If an exhaustible iterator is passed as an argument to `cycle`, the iterator won't ever exhaust - `cycle` will manage to return back to the start of the iterator and keep on cycling.

##### Example 1

In [12]:
g = cycle(('red', 'green', 'blue'))
list(islice(g, 10))

['red', 'green', 'blue', 'red', 'green', 'blue', 'red', 'green', 'blue', 'red']

##### Example 2

To show that an exhaustible iterator can be passed to `cycle` and not be exhausted:

In [14]:
def colours():
    yield 'red'
    yield 'green'
    yield 'blue'

g = cycle(colours())
list(islice(g, 10))

['red', 'green', 'blue', 'red', 'green', 'blue', 'red', 'green', 'blue', 'red']

##### Example 3

A more real-life example of `cycle` could be if you had a card deck that you want to deal out to 4 players. 

In [22]:
from collections import namedtuple

Card = namedtuple('Card', 'rank suit')

def card_deck():
    RANKS = tuple(str(i) for i in range(2, 11)) + tuple('JQKA')
    SUITS = ('Spades', 'Hearts', 'Diamonds', 'Clubs')

    for suit in SUITS:
        for rank in RANKS:
            yield Card(rank, suit)

hands = [list() for _ in range(4)]

index_cycle = cycle([0, 1, 2, 3])
for card in card_deck():
    hands[next(index_cycle)].append(card)

hands

But we can improve upon this...

The `index.cycle = cycle([0, 1, 2, 3])` is just cycling through like so: `hands[0]`, `hands[1]`, `hands[2]`, `hands[3]`, `hands[0]`, ...

So what we're actually doing is just cycling through the `hands` iterable.

Why not make `hands` into a cycle object?

In [24]:
from collections import namedtuple

Card = namedtuple('Card', 'rank suit')

def card_deck():
    RANKS = tuple(str(i) for i in range(2, 11)) + tuple('JQKA')
    SUITS = ('Spades', 'Hearts', 'Diamonds', 'Clubs')

    for suit in SUITS:
        for rank in RANKS:
            yield Card(rank, suit)

hands = [list() for _ in range(4)]
hands_cycle = cycle(hands)

for card in card_deck():
    next(hands_cycle).append(card)

hands

[[Card(rank='2', suit='Spades'),
  Card(rank='6', suit='Spades'),
  Card(rank='10', suit='Spades'),
  Card(rank='A', suit='Spades'),
  Card(rank='5', suit='Hearts'),
  Card(rank='9', suit='Hearts'),
  Card(rank='K', suit='Hearts'),
  Card(rank='4', suit='Diamonds'),
  Card(rank='8', suit='Diamonds'),
  Card(rank='Q', suit='Diamonds'),
  Card(rank='3', suit='Clubs'),
  Card(rank='7', suit='Clubs'),
  Card(rank='J', suit='Clubs')],
 [Card(rank='3', suit='Spades'),
  Card(rank='7', suit='Spades'),
  Card(rank='J', suit='Spades'),
  Card(rank='2', suit='Hearts'),
  Card(rank='6', suit='Hearts'),
  Card(rank='10', suit='Hearts'),
  Card(rank='A', suit='Hearts'),
  Card(rank='5', suit='Diamonds'),
  Card(rank='9', suit='Diamonds'),
  Card(rank='K', suit='Diamonds'),
  Card(rank='4', suit='Clubs'),
  Card(rank='8', suit='Clubs'),
  Card(rank='Q', suit='Clubs')],
 [Card(rank='4', suit='Spades'),
  Card(rank='8', suit='Spades'),
  Card(rank='Q', suit='Spades'),
  Card(rank='3', suit='Hearts'),


#### `itertools.repeat`

This function simply yields the same value indefinitely, but an additional argument can be specified to make the count finite.

##### Example 1

In [28]:
from itertools import repeat

g = repeat('Python')

for _ in range(3):
    print(next(g))

Python
Python
Python


In [29]:
g = repeat('Python', 4)
list(g)

['Python', 'Python', 'Python', 'Python']

**One important thing to note:** The items yielded by `repeat` are the **same** object. So, if that object is a mutable and it is mutated between repeats, that mutation will be observed.

# 05 - Chaining and Teeing Iterators

#### `chain(*args)` and `chain.from_iterable(it)` 

`itertools.chain(*args)` takes a number of args that can be **iterables** and returns a lazy **iterator**.

`chain` is analogous to sequence concatenation. To implement the functionality of `chain` without using `chain`, you can do:

```python
for it in (iter1, iter2, iter3):
    yield from it
```

This will `yield` everything from `iter1` before moving onto `iter2`, and then finally from `iter3`.

With `chain` you will get:

```python
l = [iter1, iter2, iter3]
for item in chain(*l):
    print(item)
```
The items are still `yield`ed but there's a caveat!

**Caveat: unpacking an iterable of iterators is _eager_ not _lazy_. So above, we have iterated through `l` (iterable) eagerly, but we haven't touched the iterators yet. If `l` was an iterable of 1 million iterators, we would have unpacked all of them eagerly before starting the loop and getting the first item out of the first iterator.** 

Instead, if we really need **lazy** evaluation, we can use `itertools.chain.from_iterable(it)`

This takes an iterable like `l` above which contains a list of iterators. Python will lazily iterate through this iterable and `yield from` the first iterator lazily of course.

In [41]:
from itertools import chain

l1 = (i**2 for i in range(4))
l2 = (i**2 for i in range(4, 8))
l3 = (i**2 for i in range(8, 12))

lists = [l1, l2, l3]

for item in chain(*lists):
    print(item)

0
1
4
9
16
25
36
49
64
81
100
121


We can observe the **eager** unpacking by storing all our iterators in a generator function.

In [44]:
def squares():
    print('yielding 1st item')
    yield (i**2 for i in range(2))
    print('yielding 2nd item')
    yield (i**2 for i in range(2, 5))
    print('yielding 3rd item')
    yield (i**2 for i in range(5, 8))


for item in chain(*squares()):
    print(item)

yielding 1st item
yielding 2nd item
yielding 3rd item
0
1
4
9
16
25
36
49


As you can see in the print statements above, all iterators are yielded before the main loop starts.

But we can get around that using `chain.from_iterable()`

In [45]:
def squares():
    print('yielding 1st item')
    yield (i**2 for i in range(2))
    print('yielding 2nd item')
    yield (i**2 for i in range(2, 5))
    print('yielding 3rd item')
    yield (i**2 for i in range(5, 8))

for item in chain.from_iterable(squares()):
    print(item)

yielding 1st item
0
1
yielding 2nd item
4
9
16
yielding 3rd item
25
36
49


#### `tee(iterable, n)`

Let's say we are sent an iterator (e.g. via a `return` of a function) that is very difficult/time-consuming to acquire. How would we make a copy of it? 

One way that we've seen is just to manually create the iterator multiple times:
```python
iters = []
for _ in range(10):
    iters.append(create_iterator())
```
but if it's time-consuming we may not want to do that.

The other solution is `itertools.tee` which returns **independent iterators in a tuple**. This can let us iterate through the same iterator **multiple times** or even **in parallel**.
```python
tee(iterable, 10) -> (iter1, iter2, ..., iter10)
```
`iter1` through `iter10` are all different objects, but they are all **lazy iterators**. *Always. Even if the original argument was not.*

```python
l = [1, 2, 3, 4]
tee(l, 3) -> (iter1, iter2, iter3)
```
`iter1` and the rest are all **lazy iterators** despite `l` being an **iterable**.

##### Example 1

Let's make 3 **independent** copies of an iterator:

In [46]:
from itertools import tee

def squares(n):
    for i in range(n):
        yield i**2

gen = squares(5)

iters = tee(gen, 3)
iters  

(<itertools._tee at 0x2732a6dbd80>,
 <itertools._tee at 0x2732a6dbe80>)

As you can see the memory addresses are different, therefore, iterating through one has no impact on the other iterators. We have *tee'd up our iterator 3 times*.

Let's tee up a list (**iterable**) multiple times and look for a return value of multiple **iterators**. 

In [51]:
l = [1, 2, 3, 4]

lists = tee(l, 2)
lists

(<itertools._tee at 0x2732a6e8e00>, <itertools._tee at 0x273291e4dc0>)

In [52]:
list(lists[0])

[1, 2, 3, 4]

`lists[0]` is now exhausted:

In [53]:
list(lists[0])

[]

In [54]:
lists[0] is iter(lists[0])

True

# 06 - Mapping and Reducing

# 07 - Zipping

# 08 - Grouping

# 09 - Combinatorics